In [1]:
# Import libraries
import pandas as pd
from pathlib import Path 


In [2]:
raw_file = Path(r"C:\Users\preci\Downloads\archive\WA_Fn-UseC_-Telco-Customer-Churn.csv")
data = Path(r"C:\Users\preci\Downloads\archive\WA_Fn-UseC_-Telco-Customer-Churn.csv")

In [3]:
# Load data
df = pd.read_csv(data)
print("shape before cleaning:", df.shape)
print(df.head())

shape before cleaning: (7043, 21)
   customerID  gender  SeniorCitizen Partner Dependents  tenure PhoneService  \
0  7590-VHVEG  Female              0     Yes         No       1           No   
1  5575-GNVDE    Male              0      No         No      34          Yes   
2  3668-QPYBK    Male              0      No         No       2          Yes   
3  7795-CFOCW    Male              0      No         No      45           No   
4  9237-HQITU  Female              0      No         No       2          Yes   

      MultipleLines InternetService OnlineSecurity  ... DeviceProtection  \
0  No phone service             DSL             No  ...               No   
1                No             DSL            Yes  ...              Yes   
2                No             DSL            Yes  ...               No   
3  No phone service             DSL            Yes  ...              Yes   
4                No     Fiber optic             No  ...               No   

  TechSupport StreamingTV St

Basic Cleaning

In [4]:
# Strip whitespace from column names
df.columns = df.columns.str.strip()

# Fix TotalCharges (common issue: blanks as strings)
if "TotalCharges" in df.columns:
    df["TotalCharges"] = pd.to_numeric(df["TotalCharges"], errors="coerce")

# Numeric columns: fill missing with median
numeric_cols = ["tenure", "MonthlyCharges", "TotalCharges"] 
for col in numeric_cols:
    if col in df.columns:
        median_val = df[col].median()
        df[col] = df[col].fillna(median_val)

# Categorical columns: fill missing with "Unknown"
cat_cols = [c for c in df.columns if c not in numeric_cols]           
for col in cat_cols:
    df[col] = df[col].fillna("Unknown")
print("After cleaning:", df.isna().sum())    

After cleaning: customerID          0
gender              0
SeniorCitizen       0
Partner             0
Dependents          0
tenure              0
PhoneService        0
MultipleLines       0
InternetService     0
OnlineSecurity      0
OnlineBackup        0
DeviceProtection    0
TechSupport         0
StreamingTV         0
StreamingMovies     0
Contract            0
PaperlessBilling    0
PaymentMethod       0
MonthlyCharges      0
TotalCharges        0
Churn               0
dtype: int64


Feature engineering (groups + churn flag)

In [5]:
# Tenure groups
def tenure_group(tenure):
    if pd.isna(tenure):
        return "Unknown"
    if tenure < 12:
        return "< 1 year"
    elif tenure < 24:
        return "1-2 years"
    elif tenure < 48: 
        return "2-4 years"
    else:
        return "4+ years"
    
df["TenureGroup"] = df["tenure"].apply(tenure_group)

# Monthly charges groups
def charges_group(charges):
    if pd.isna(charges):
        return "Unknown"
    if charges < 30:
        return "< $30"
    elif charges < 60: 
        return "$30-$60"
    elif charges < 90:
        return "$60-$90"
    else:
        return "$90+"
    
df["MonthlyChargesGroup"] = df["MonthlyCharges"].apply(charges_group) 

# Churn flag 0/1
df["ChurnFlag"] = df["Churn"].map({"Yes": 1, "No": 0}).fillna(0).astype(int)
df.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn,TenureGroup,MonthlyChargesGroup,ChurnFlag
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,Month-to-month,Yes,Electronic check,29.85,29.85,No,< 1 year,< $30,0
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,No,One year,No,Mailed check,56.95,1889.50,No,2-4 years,$30-$60,0
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes,< 1 year,$30-$60,1
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,No,One year,No,Bank transfer (automatic),42.30,1840.75,No,2-4 years,$30-$60,0
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes,< 1 year,$60-$90,1


Export cleaned data for SQL/Power BI

In [6]:
data = Path(r"C:\Users\preci\Downloads\archive\TelcoCustomerChurn_Clean.csv")
df.to_csv(data, index=False)
data

WindowsPath('C:/Users/preci/Downloads/archive/TelcoCustomerChurn_Clean.csv')